In [1]:
%%time
# Import necessary Libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import pickle # used to store cleaned data
import numpy.ma as ma # masking for numpy Arrays
boolVal = False
attedDataName = '../../project_data/Overlapping_GenesMR.p'
bioGRIDfileName = '../../project_data/BioGRID_with_ATTEDMR.p'
if boolVal:
    # Reading in BioGRID as DataFrame
    path = os.path.join('C:\\Users\\ysman\\OneDrive\\Desktop\\project_data\\BIOGRID-ORGANISM-Arabidopsis_thaliana_Columbia-3.5.181.tab2.txt')
    bioGRID_file = open(path, "r") 
    testFile = open(os.path.join('C:\\Users\\ysman\\OneDrive\\Desktop\\project_directory\\data\\test.txt'), 'r')
    bioGRID_DF = pd.read_csv(bioGRID_file, sep = '\t')
    # Simplified DataFrame to only include interactions
    simplebGRID = bioGRID_DF[['Entrez Gene Interactor A','Entrez Gene Interactor B']]
    # Identifying Organisms Present in BioGRID
    OrganismTypesA = list(bioGRID_DF['Organism Interactor A'].unique())
    OrganismTypesB = list(bioGRID_DF['Organism Interactor B'].unique())
    OrganismTypesA.sort()
    OrganismTypesB.sort()
    OrganismTypes = list(set([*OrganismTypesA, *OrganismTypesB]))
    OrganismTypes.sort()
    # Use NCBI's Taxonomy Name/ID Status Report - Plug in OrganismTypes and get .txt
    organismIDs = pd.read_csv('../../project_data/tax_report.txt', sep = '\t')
    organismIDs.drop(columns = ['|','|.1','|.2', 'code', 'primary taxid'],inplace = True)
    # Categorize into different subsets based on organism ID. We know '3702' is Arabidopsis
    mask1 = bioGRID_DF['Organism Interactor A'] == 3702
    mask2 = bioGRID_DF['Organism Interactor B'] == 3702
    onlyArabDF = bioGRID_DF[mask1& mask2]
    oneArabDF = bioGRID_DF[~mask1|~mask2]
    noArabDF = bioGRID_DF[~mask1 & ~mask2]
    # Get list of genes so I can import the necessary ATTED Data. Note that the ATTED data has a text file by Entrez gene ID
    # WholeData:
    wholeGenesA = list(bioGRID_DF['Entrez Gene Interactor A'].unique())
    wholeGenesB = list(bioGRID_DF['Entrez Gene Interactor B'].unique())
    wholeGenesA.sort()
    wholeGenesB.sort()
    wholeGenes = list(set([*wholeGenesA, *wholeGenesB]))
    wholeGenes.sort()
    # Only Arabidopsis Subset
    ArabGenesA = list(onlyArabDF['Entrez Gene Interactor A'].unique())
    ArabGenesB = list(onlyArabDF['Entrez Gene Interactor B'].unique())
    ArabGenesA.sort()
    ArabGenesB.sort()
    ArabGenes = list(set([*ArabGenesA, *ArabGenesB]))
    ArabGenes.sort()
    # Read in only the Overlapping Genes
    # Get a list of all genes in ATTED
    atted = pd.read_csv('../../project_data/Ath-mB.v17-08.G20819-S16033.rma_combat.mrgeo.d/814630', sep = '\t', header = None)
    atted = atted.sort_values(by = 0)
    a1 = np.array(atted[0])
    attedGenes = list(a1)
    # Reading in of Overlapping Genes
    attedpath = 'C:\\Users\\ysman\\OneDrive\\Desktop\\project_data\\Ath-mB.v17-08.G20819-S16033.rma_combat.mrgeo.d\\'
    overlapGenes = []
    for i in range (len(wholeGenes)):
        if os.path.exists(attedpath+'{}'.format(wholeGenes[i])):
            overlapGenes.append(wholeGenes[i])
    DF = {0:attedGenes}
    for x in overlapGenes:
        tempAtted = pd.read_csv('../../project_data/Ath-mB.v17-08.G20819-S16033.rma_combat.mrgeo.d/{}'.format(x), sep = '\t', header= None)
        tempAtted = tempAtted.sort_values(by = 0)
        templist = list(tempAtted[1])
        DF.update({x:templist})
    attedData = pd.DataFrame(DF,dtype='float64')
    attedData = attedData.astype({0:'int'})
    attedData = attedData.set_index(0)
    pickle.dump(attedData, open(attedDataName, 'wb'))
    # Add ATTED MR Values to BioGRID
    genesGRID = simplebGRID.loc[:]
    genesGRID['MR']=np.zeros(len(genesGRID))
    # for gA,gB in zip(toyGRID.iloc[:,0],toyGRID.iloc[:,1]): is a way to iterate through 2 cols at once
    for i in range(genesGRID.shape[0]):
        gA = genesGRID.loc[i,'Entrez Gene Interactor A']
        gB = genesGRID.loc[i,'Entrez Gene Interactor B']
        if (gA in attedData.columns) and (gB in attedData.columns): # checks if both genes are part of the overlapping set
            mr = attedData.loc[gA,gB]
            genesGRID.loc[i,'MR'] = mr
        else:
            genesGRID.loc[i,'MR'] = np.nan
    pickle.dump(genesGRID, open(bioGRIDfileName, 'wb'))
    boolVal = False
else:
    # Load data using pickle
    attedData = pickle.load(open(attedDataName, 'rb'))
    genesGRID = pickle.load(open(bioGRIDfileName, 'rb'))
# Initializes Networkx Graph
genesG = nx.Graph()
# Adds edges for each interaction
for i in range(genesGRID.shape[0]):
    genesG.add_edge(genesGRID['Entrez Gene Interactor A'].loc[i], genesGRID['Entrez Gene Interactor B'].loc[i], weight = genesGRID['MR'].loc[i])

Wall time: 6.48 s


In [2]:
def nonConvertible(orig,trans): 
    '''
    Checks if all orginal values are in new list of values 
    and returns the ones that aren't
    Inputs: List of original values, list of new values
    Output: List of all original values that are not in the new value list
    ''' 
    nc = []
    for i in range(len(orig)):
        if orig[i] not in trans:
            nc.append(orig[i])
        else:
            continue
    return nc

In [3]:
# Read In Data 
ABC_trans = pd.read_csv('../data/convABC_Genes.txt', sep = '\t')
ABC_trans = ABC_trans.rename(columns = {'From':'TAIR_ID','To':'ENTREZ_ID'})
ABC_orig = pd.read_excel('../data/ABC_Genes.xls', sheet_name = 'Sheet2')
abcAT = sorted(set(list(ABC_trans['TAIR_ID'])))
abcEntrez = sorted(set(list(ABC_trans['ENTREZ_ID'])))
abcAT_orig = sorted(set(list(ABC_orig['TAIR_ID'])))
abc_NC  = nonConvertible(abcAT_orig,abcAT)
#print('The non-convertable ABC TAIR-IDs are: {}.'.format(', '.join(abc_NC)))
print(f'The non-convertable ABC TAIR IDs are: {abc_NC}')

LTP_trans = pd.read_csv('../data/convLTP_Genes.txt', sep = '\t')
LTP_trans = LTP_trans.rename(columns = {'From':'TAIR_ID','To':'ENTREZ_ID'})
LTP_orig = pd.read_excel('../data/LTP_Genes.xlsx', sheet_name = 'Sheet1')
ltpAT = sorted(set(list(LTP_trans['TAIR_ID'])))
ltpEntrez = sorted(set(list(LTP_trans['ENTREZ_ID'])))
ltpAT_orig = sorted(set(list(LTP_orig['TAIR_ID'])))
ltp_NC  = nonConvertible(ltpAT_orig,ltpAT)
#print('The non-convertable LTP TAIR-IDs are: {}.'.format(', '.join(ltp_NC)))
print(f'The non-convertable LTP TAIR IDs are: {ltp_NC}')

print(f'Total ABC Genes: {len(abcEntrez)}')
print(f'Total LTP Genes: {len(ltpEntrez)}')

The non-convertable ABC TAIR IDs are: ['ATMG00110', 'ATMG00900']
The non-convertable LTP TAIR IDs are: []
Total ABC Genes: 129
Total LTP Genes: 85


In [4]:
def existing(lst, reference): 
    '''
    Checks if items of lst are in reference, returns a list of existing values
    Inputs: List to check, list to cross-reference with
    Output: List of all values that also exist in the reference list.
    ''' 
    exists = [] # stores the existing values
    NE = [] # stores the non-existing values
    for x in lst:
        if x in reference:
            exists.append(x)
        else:
            NE.append(x)
    return exists, NE

In [13]:
def ShortestDistances(Graph,grpA,grpB,dataFrame = True): #using function caused Kernel to crash due to memory
    '''
    Finds the shortest distances in a graph between all members of group 1 
    and all members of group 2 and stores into a dict.
    Inputs: Graph to scan, list of group A nodes, list of group B nodes
    Output: Returns a pandas dataFrame with group A as columns and group B as rows 
            and the shortest-distance as the values. 
            Else, returns a dict with group A as keys and a list of shortest distances 
            as values (size of list is the length of group B)
    '''
    graphGenes = list(Graph.nodes)
    abc, Uabc = existing(grpA,graphGenes)
    ltp, Ultp = existing(grpB,graphGenes)
    DF={0:ltp}
    for x in abc:
        valList = []
        for y in ltp:
            if (x in graphGenes) and (y in graphGenes): # checks if both genes are part of the overlapping set
                val = nx.astar_path_length(Graph, x,y)
            else:
                val = np.nan
            valList.append(val)
        DF.update({x:valList})
    if dataFrame:
        Data = pd.DataFrame(DF,dtype='float64')
        Data = Data.astype({0:'int'})
        Data = Data.set_index(0)
        return Data
    else:
        return DF

In [6]:
# option: filter out nodes with 1 connection
# store as a dictionary, dict: 
def simplifyGraph(grpA, grpB, Graph, kdeg):
    '''
    Gathers all nodes within kdeg degrees of separation from the given groups.
    Inputs: list of group A nodes, list of group B nodes, graph to simplify, int representing
            the degree of seperation from the key nodes allow.
    Output: subgraph of inputted graph as well as dictionary with each node in the new graph as key
            and # of neighboring nodes in group A, # of neighboring nodes in group B, and 
            list of all neighboring nodes.
    '''
    keyGenes = sorted(set([*grpA,*grpB]))
    i = kdeg
    while i >0:
        # Stores neighbors at each key gene
        neighbors = []
        # Loops through all key genes 
        for gene in keyGenes:
            nodes = [n for n in Graph.neighbors(gene)]
            neighbors.append(nodes)
        neighbors.append(keyGenes)
        neighbors = [item for sublist in neighbors for item in sublist]
        keyGenes = sorted(set(neighbors))
        i-=1
    # Creates subgraph
    newGraph = Graph.subgraph(keyGenes)
    # Iterates through the new nodes and checks if neighbors are in either group A or B
    geneDict = {}
    for gene in newGraph.nodes:
        nodes = [n for n in newGraph.neighbors(gene)]
        grpACount = 0
        grpBCount = 0
        for ngene in nodes:
            if ngene in grpA:
                grpACount +=1
            elif ngene in grpB:
                grpBCount +=1
        geneDict[gene] = (grpACount,grpBCount, nodes)
    return newGraph, geneDict

In [7]:
abc, Uabc = existing(abcEntrez,genesG.nodes)
ltp, Ultp = existing(ltpEntrez,genesG.nodes)
print(f'The original Graph has {len(genesG.nodes)} nodes and {len(genesG.edges)} edges.')
print(f'Graph-compatible ABC Genes: {len(abc)}/{len(abcEntrez)}')
print(f'Graph-compatible LTP Genes: {len(ltp)}/{len(ltpEntrez)}')

The original Graph has 10550 nodes and 48981 edges.
Graph-compatible ABC Genes: 75/129
Graph-compatible LTP Genes: 14/85


In [8]:
simpleG, simpleGDict = simplifyGraph(abc,ltp, genesG,1)
print(f'The simplified Graph has {len(simpleG.nodes)} nodes and {len(simpleG.edges)} edges.')

The simplified Graph has 399 nodes and 2907 edges.
